In [7]:
import pandas as pd
import re
import os
from tqdm import tqdm
import sys
root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(root_path)
from src.utils import constants
import spacy
from typing import Dict, Set, Tuple, Optional, List, Any

In [22]:
DATASET_FOLDER = 'E:/AI ML DL/Amazon_ML_Challenge/data/raw/dataset/'
TRAIN_PATH = os.path.join(DATASET_FOLDER, 'train.csv')
TEST_PATH = os.path.join(DATASET_FOLDER, 'test.csv')
SAMPLE_TEST_PATH = os.path.join(DATASET_FOLDER, 'sample_test.csv')
SAMPLE_TEST_OUT_PATH = os.path.join(DATASET_FOLDER, 'sample_test_out.csv')

In [23]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
sample_test = pd.read_csv(SAMPLE_TEST_PATH)
sample_test_out = pd.read_csv(SAMPLE_TEST_OUT_PATH)

In [ ]:
sample_test.entity_name.head(), sample_test_out.prediction.head()

In [19]:
import cv2
import csv
from paddleocr import PaddleOCR

In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang='en')

In [30]:
OUTPUT_CSV = os.path.join(DATASET_FOLDER, 'ocr_results_test.csv')

In [ ]:
with open(OUTPUT_CSV, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["image_link", "extracted_text"])  # Header row

    IMAGES_TEST_DIR = 'E:/AI ML DL/Amazon_ML_Challenge/data/raw/images/train'
    # Loop through each image in the directory
    for image_name in os.listdir(IMAGES_TEST_DIR):
        if image_name.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):  # Filter for image files
            image_path = os.path.join(IMAGES_TEST_DIR, image_name)

            # Read the image
            img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

            # Perform OCR on the image
            try:
                res = ocr.ocr(img, cls=True)
                if res[0] is None:
                    writer.writerow([image_path, None])
                    continue

                # Extract all text from the image
                extracted_text = ""
                for line in res:
                    for word_info in line:
                        extracted_text += word_info[-1][0] + " "  # Concatenate all the words

                # Write the image path and extracted text to the CSV file
                writer.writerow([image_path, extracted_text.strip()])
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
                continue
            

In [ ]:
df = pd.read_csv(TEST_PATH)
ou = pd.read_csv(OUTPUT_CSV)

# Preprocessing and mapping the extracted data to the dataframe
df['image_link'] = df['image_link'].apply(lambda x: x.split('/')[-1])
df.drop(columns=['group_id'], inplace=True)

ou['image_link'] = ou['image_link'].apply(lambda x: x.split('/')[-1])
ou['image_link'] = ou['image_link'].apply(lambda x: x.split('\\')[-1])

# Merge the test dataframe and OCR results based on image links
ou = ou.merge(df, on=['image_link'], how='right')
ou.drop(columns=['image_link'], inplace=True)